In [1]:
# Problema da mochila 0-1
import sys
import numpy as np
import gurobipy as gb
from gurobipy import GRB

In [13]:
# resolve o MIP
def mip (n, p, w, c):
    
    # cria modelo
    model = gb.Model()
    
    # desabilita print do solver
    model.Params.LogToConsole = 0 
    
    # define variáveis
    x = model.addVars(n, vtype=GRB.BINARY, name='x')
    
    obj = None
    for i in range(0, n):
        obj += p[i] * x[i]
 
    model.setObjective(obj, GRB.MAXIMIZE)

    constr = None
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr <= c)
    
    # define restrições
    #model.addConstr((x.prod(w)) <= c, name= 'mochila')
    
    # define função objetivo
    #model.setObjective(x.prod(p), GRB.MAXIMIZE)
    
    # resolve o problema
    model.optimize()
    
    # pegando solução
    xopt = []
    for v in model.getVars():
        xopt.append(v.x)
        
    print("x = ", np.round(xopt, 2))

    return model.objVal

In [14]:
# resolve problema da mochila 0-1 relaxado
def lp(n, p, w, c):

    v = np.divide(p, w) # p/w
    #print("v = ", np.round(v,2))
    
    # indices de v em ordem não-decrescente
    idx = np.argsort(v, kind='quicksort')[::-1]
    #print("idx = ", idx)
    
    # faz cópia da capacidade
    aux = c 
    
    # cria vetor solução
    x = np.zeros(n)

    for i in idx:
        if aux >= w[i]: # verifica se existe espaço na mochila
            aux = aux - w[i] # atualiza a capacidade da mochila
            x[i] = 1 # atualiza a solução
        else:
            xh = aux/w[i] # calculo da proporção que cabe na mochila
            x[i] = xh # atualiza a solução
            break
            
    # valor ótimo
    opt = sum(p*x)
    
    print("x = ", np.round(x,2))
    
    return opt 

In [15]:
def primal(n, p, w, c):

    v = np.divide(p, w) # p/w
    
    # indices ordenação
    idx = np.argsort(v, kind='quicksort')[::-1]
    
    # copia da capacidade
    aux = c
    
    # cria vetor solução
    x = np.zeros(n)

    for i in idx:
        if aux >= w[i]: # verifica se tem espaço na mochila
            aux = aux - w[i] # atualiza a capacidade
            x[i] = 1 # atualiza a solução
        else:
            x[i] = 0 # atualiza a solução
            
    opt = sum(p*x)
    
    print("x = ", x)
    
    return opt

In [16]:
# resolve o problema com programação dinâmica
def dinamica(n,c,a,b,x):
    
    f = np.zeros((n,b+1), dtype=int)
    pt = np.zeros((n,b+1), dtype=int)
    
    # calculando o valor ótimo
    for j in range(b+1):
        if (j < a[0]):
            f[0][j] = 0
        if (j >= a[0]):
            if (c[0] >= 0):
                f[0][j] = c[0]
                pt[0][j] = 1
            else:
                f[0][j] = 0
                pt[0][j] = 0
                    
    for i in range(1,n):
        f[i][0] = 0
        for j in range(b+1):
            if (a[i] <= j):
                if (c[i]+f[i-1][j-a[i]] > f[i-1][j]):
                    f[i][j] = c[i]+f[i-1][j-a[i]]
                else:
                    f[i][j] = f[i-1][j]
            else:
                f[i][j] = f[i-1][j]
            sol = f[i][j]
    
    # vetor a ser usado para se encontrar a solução
    for i in range(1,n):
        for j in range(b+1):
            if (f[i][j] == f[i-1][j]):
                pt[i][j] = 0
            else:
                pt[i][j] = 1
    
    # recuperando a solução x
    aux = b
    for i in range(n-1,-1,-1):
        if (aux <= 0):
            break
        if (pt[i][aux] == 0):
            x[i] = 0
        if (pt[i][aux] == 1):
            x[i] = 1
            aux -= a[i]
    
    print("x:", np.round(x,2))
            
    return sol

In [17]:
if __name__ == "__main__":
    
        # instância
#    p = [6, 5, 8, 9, 6, 7, 3] # vetor de lucros
#    w = [2, 3, 6, 7, 5, 9, 3] # vetor de pesos
#    c = 9 # capacidade

    # número de itens
#    n = len(p) 

    # variável 
    #x = np.zeros(n, dtype = int)
    
    datafile = "instances/kp/10/10_100_1.txt"
    
    with open(datafile, 'r') as file: linhas = file.readlines()
        
    linhas = [a.strip() for a in linhas] # remove linha vazia inicial e elimina os "\n" de cada linha

    n = int(linhas[0]) # ler o tamanho da instancia
  
    d = np.fromstring(linhas[1], dtype=int, sep = ' ') # ler a diagonal da matriz
  
    p = np.zeros((n), dtype=int) # define o vetor de lucro

    for i in range(n): # preenche a diagonal
        p[i] = d[i]

    c = int(linhas[2]) # ler a capacidade

    w = np.fromstring(linhas[3], dtype=int, sep = ' ') # ler os pesos
    
    # resolve mip
    print("Soluçao do MIP")
    opt = mip (n, p, w, c)
#    print("valor ótimo =", np.round(opt,2))
#    print("\n")
    
    # resolve relaxação linear
    print("Solução da relaxação linear")
    opt = lp (n, p, w, c)
    print("valor ótimo = ", np.round(opt,2))
    print("\n")

    # resolve relaxação linear
    print("Solução primal")
    opt = primal (n, p, w, c)
    print("valor ótimo = ", np.round(opt,2))
    print("\n")
    
    # resolve usando programação dinâmica
    print("Solução da programação dinâmica")
    opt = dinamica(n,p,w,c,x)
    print("valor ótimo = ", np.round(opt,2))

Soluçao do MIP
x =  [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
Solução da relaxação linear
x =  [1.   1.   1.   1.   1.   1.   1.   0.17 1.   1.  ]
valor ótimo =  308.33


Solução primal
x =  [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
valor ótimo =  306.0


Solução da programação dinâmica
x: [1 1 1 1 1 1 1 0 1 1]
valor ótimo =  306
